In [1]:
from tools4rdf.network.network import OntologyNetwork
from rdflib import Graph, Namespace, RDFS, RDF, URIRef, Namespace, Literal, XSD
import pandas as pd
from uuid import uuid4

In [2]:
def get_id(term):
    return term.strip() + ":" + str(uuid4()).split("-")[0]

In [3]:
onto = OntologyNetwork("pizza.owl")

In [4]:
onto.terms.pizza.hasSpiciness

pizza:hasSpiciness

In [5]:
df = pd.read_csv("pizza_kg.csv")

In [6]:
df

,PIZZA,PIZZABASE,PIZZATOPPING1,CALORIFICCONTENTVALUE,PIZZATOPPING2,CALORIFICCONTENTVALUE2,PIZZATOPPING3,CALORIFICCONTENTVALUE3,PIZZATOPPING4,CALORIFICCONTENTVALUE4,FOOD,PRICE,SPICINESS
0,IceCream,-,-,-,-,-,-,-,-,-,Milk,7.00,-
1,American,PizzaBase,MozzarellaTopping,280,PeperoniSausageTopping,494,TomatoTopping,18,-,-,-,10.00,-
2,AmericanHot,PizzaBase,MozzarellaTopping,280,PeperoniSausageTopping,494,TomatoTopping,18,JalapenoPepperTopping,29,-,11.50,Hot
3,Margherita,DeepPanBase,MozzarellaTopping,280,-,-,TomatoTopping,18,-,-,-,9.50,-
4,Margherita,ThinAndCrispyBase,MozzarellaTopping,280,-,-,TomatoTopping,18,-,-,-,8.99,-
5,Mushroom,PizzaBase,MozzarellaTopping,280,MushroomTopping,22,TomatoTopping,18,-,-,-,11.00,-
6,SpicyPizza,PizzaBase,MozzarellaTopping,280,SweetPepperTopping,26,TomatoTopping,18,-,-,-,12.99,Mild
7,SpicyPizza,PizzaBase,MozzarellaTopping,280,HotGreenPepperTopping,40,TomatoTopping,18,-,-,-,11.00,Hot
8,VegetarianPizza,PizzaBase,MozzarellaTopping,280,ArtichokeTopping,47,TomatoTopping,18,OliveTopping,115,-,11.50,-


In [9]:
g = Graph()
nm = Namespace('https://example.org/pizza#')

In [10]:
termdict = {}

for index, row in df.iterrows():
    pid = get_id(str(row["PIZZA"]))
    term = URIRef(pid)
    g.add((term, RDF.type, getattr(nm, str(row["PIZZA"]))))

    #add base
    pizzabase = str(row["PIZZABASE"])
    if pizzabase != "-":
        if pizzabase in termdict.keys():
            baseid = termdict[pizzabase]
        else:
            baseid = URIRef(get_id(pizzabase))
            termdict[pizzabase] = baseid
            g.add((baseid, RDF.type, getattr(nm, pizzabase)))
        
        g.add((term, nm.hasBase, baseid))
        

    #add toppings
    toppings = ["PIZZATOPPING1", "PIZZATOPPING2", "PIZZATOPPING3", "PIZZATOPPING4"]
    calories = ["CALORIFICCONTENTVALUE", "CALORIFICCONTENTVALUE2", "CALORIFICCONTENTVALUE3", "CALORIFICCONTENTVALUE4"]
    for (topping, calorie) in zip(toppings, calories):
        topbase = str(row[topping])
        if topbase != "-":
            if topbase in termdict.keys():
                topid = termdict[topbase]
            else:
                topid = URIRef(get_id(topbase))
                termdict[topbase] = topid
                g.add((topid, RDF.type, getattr(nm, topbase)))
            g.add((term, nm.hasTopping, topid)) 
        
        calbase = str(row[calorie])
        if calbase != "-":
            g.add((getattr(nm, topbase), nm.hasCalorificContentValue, Literal(float(calbase), datatype=XSD.float)))  

    #add base
    food = str(row["FOOD"])
    if food != "-":
        if food in termdict.keys():
            foodid = termdict[food]
        else:
            foodid = URIRef(get_id(food))
            termdict[food] = foodid
            g.add((foodid, RDF.type, getattr(nm, food)))
        g.add((term, nm.hasIngredient, foodid))

    #add base
    price = str(row["PRICE"])
    if price != "-":
        g.add((term, nm.hasPrice, Literal(float(price), datatype=XSD.float)))

    #add base
    spice = str(row["SPICINESS"])
    if spice != "-":
        if spice in termdict.keys():
            spiceid = termdict[spice]
        else:
            spiceid = URIRef(get_id(spice))
            termdict[spice] = spiceid
            g.add((spiceid, RDF.type, getattr(nm, spice)))
        g.add((term, nm.hasSpiciness, spiceid))

In [11]:
g.serialize("pizza_kg.ttl", format="turtle")

<Graph identifier=Nbd2929ffbb1547e1b3cc15ab66477a4a (<class 'rdflib.graph.Graph'>)>